<a href="https://colab.research.google.com/github/ryanjschwarz/summary/blob/main/Pegasus_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install dependencies

In [ ]:
# Install PyTorch
!pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install sentencepiece

In [ ]:
# Install transformers
!pip install transformers

In [ ]:
#Install Beautiful soup
!pip install Beautifulsoup4
!pip install requests

# 1. Import and Load Model

In [6]:
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

#Import Scraper
from bs4 import BeautifulSoup as bs
import requests
import re

In [ ]:
# Load tokenizer 
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-wikihow") #Need to try different pretrained data sets

In [ ]:
# Load model 
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-wikihow")

# 2. Scrape for question and answers

In [92]:
#Scrape post
post_link = 'https://stackoverflow.com/questions/53621696/how-to-start-from-second-index-for-for-loop/53621801'
link = post_link
results = requests.get(link)
doc = bs(results.text, 'html.parser')

#This parses specifically for question text
text_tags = doc.find_all('div', itemprop='text')#, itemprop='text')

if (len(text_tags[0].text.split('\r')) == 1): #If there is not community lock message the question is the first element in the list
  question_text = text_tags[0].text.split('\r')[0] #the actual text is the 3rd element in the html tag

else:
  question_text = text_tags[0].text.split('\r')[2] #If there is a community lock message it will be the third element for some reason

##Strips random HTML and \n characters out of the text_tag
question_text = question_text.replace('\n',' ')
question_text = question_text.strip()

#question_text
answer_texts = []
#This parses for the various answer text_tags and strips out random characters
for i in range(1, len(text_tags)):
    answer_texts.append(text_tags[i].text)
    answer_texts[i-1] = answer_texts[i-1].replace('\n', ' ')
    answer_texts[i-1] = answer_texts[i-1].strip()

answer_texts
    

["First thing is to remember that python uses zero indexing. You can iterate throught the list except using the range function to get the indexes of the items you want or slices to get the elements. What I think is becoming confusing here is that in your example, the values and the indexes are the same so to clarify I'll use this list as example: I = ['a', 'b', 'c', 'd', 'e'] nI = len(I) # 5  The range function will allow you to iterate through the indexes: for i in range(1, nI):     print(i) # Prints: # 1 # 2 # 3 # 4  If you want to access the values using the range function you should do it like this: for index in range(1, nI):     i = I[index]     print(i) # Prints: # b # c # d # e  You can also use array slicing to do that and you don't even need nI. Array slicing returns a new array with your slice. The slice is done with the_list_reference[start:end:steps] where all three parameters are optional and: start is the index of the first to be included in the slice end is the index of 

In [82]:
text = answer_texts[0]

#3. Perform Abstractive Summarization

In [83]:
# Create tokens - number representation of our text
tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")

In [ ]:
# Input tokens
#tokens

In [85]:
# Summarize 
summary = model.generate(**tokens)

In [90]:
# Output summary tokens
#summary[0]

In [88]:
# Decode summary
tokenizer.decode(summary[0])

'This article relies on the previous accomplishment of the three articles preceding it:<n><n>How to Create a Spirallic Spin Particle Path or Necklace Form or Spherical Border<n>How to Create a Spirallic Spin Particle Path or Necklace Form or Spherical Border<n>How to Create a Spirallic Spin Particle Path or Necklace Form or Spherical Border.'